In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data

In [101]:
# Read data from files 
train = pd.read_csv( "/Users/xtreemtg888/Downloads/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "/Users/xtreemtg888/Downloads/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "/Users/xtreemtg888/Downloads/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [104]:
train.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [12]:
# Import various modules for string cleaning


def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [15]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [16]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:302: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:375: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use 

In [71]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [28]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [32]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

2019-12-26 11:23:32,578 : INFO : 'pattern' package not found; tag filters are not available for English
2019-12-26 11:23:32,585 : INFO : collecting all words and their counts
2019-12-26 11:23:32,586 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-26 11:23:32,657 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-12-26 11:23:32,707 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-12-26 11:23:32,759 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-12-26 11:23:32,812 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types

Training model...



2019-12-26 11:23:32,862 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-12-26 11:23:32,925 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-12-26 11:23:32,975 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-12-26 11:23:33,029 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-12-26 11:23:33,077 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-12-26 11:23:33,140 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-12-26 11:23:33,199 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-12-26 11:23:33,247 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keeping 54119 word types
2019-12-26 11:23:33,303 : INFO : PROGRESS: at sentence #130000, processed 2894303 words, kee

2019-12-26 11:23:36,653 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2019-12-26 11:23:36,707 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2019-12-26 11:23:36,774 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2019-12-26 11:23:36,822 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2019-12-26 11:23:36,855 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2019-12-26 11:23:36,856 : INFO : Loading a fresh vocabulary
2019-12-26 11:23:36,931 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2019-12-26 11:23:36,932 : INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2019-12-26 11:23:36,989 : INFO : deleting the raw counts dictionary of 123504 items
2019-12-26 11:23:36,993 : I

2019-12-26 11:24:26,501 : INFO : EPOCH 4 - PROGRESS: at 19.81% examples, 832300 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:27,507 : INFO : EPOCH 4 - PROGRESS: at 26.80% examples, 844745 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:28,523 : INFO : EPOCH 4 - PROGRESS: at 33.83% examples, 850282 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:29,523 : INFO : EPOCH 4 - PROGRESS: at 40.69% examples, 855126 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:30,532 : INFO : EPOCH 4 - PROGRESS: at 47.62% examples, 858709 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:31,537 : INFO : EPOCH 4 - PROGRESS: at 54.37% examples, 858907 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:32,541 : INFO : EPOCH 4 - PROGRESS: at 60.97% examples, 857634 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:33,542 : INFO : EPOCH 4 - PROGRESS: at 67.63% examples, 856953 words/s, in_qsize 7, out_qsize 0
2019-12-26 11:24:34,542 : INFO : EPOCH 4 - PROGRESS: at 73.17% examples, 843492 words/s, in_qsiz

In [33]:
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-12-26 11:30:51,988 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2019-12-26 11:30:51,995 : INFO : not storing attribute vectors_norm
2019-12-26 11:30:52,008 : INFO : not storing attribute cum_table
2019-12-26 11:30:52,587 : INFO : saved 300features_40minwords_10context


In [34]:
model.most_similar("man")

/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6165990233421326),
 ('lady', 0.5789034366607666),
 ('lad', 0.5741980075836182),
 ('chap', 0.550387978553772),
 ('farmer', 0.5432334542274475),
 ('guy', 0.5413432717323303),
 ('person', 0.5195626616477966),
 ('men', 0.5081881880760193),
 ('businessman', 0.5043144822120667),
 ('boy', 0.5005319714546204)]

In [38]:
model['flower']

/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.12994444,  0.1057947 , -0.08783261,  0.04661154, -0.00945111,
        0.01738379, -0.02683215, -0.14264965,  0.00086006, -0.04614497,
        0.01462997,  0.04672693,  0.02298148, -0.08324099,  0.13758564,
       -0.13810883,  0.0377893 , -0.09433182,  0.07784224, -0.02788159,
       -0.024063  ,  0.0494056 ,  0.02090591, -0.13544355,  0.08008441,
        0.04486235,  0.18030944, -0.13437366, -0.02127176,  0.02761249,
       -0.06890727,  0.00545235,  0.02160632,  0.07174897, -0.04417557,
        0.00560483, -0.02126673,  0.11715391, -0.01904276,  0.06660108,
       -0.01691955,  0.03454748, -0.01770631,  0.11472204, -0.10545207,
        0.08491798, -0.00060603,  0.00141504, -0.01782763, -0.08568326,
       -0.05576062,  0.01464   , -0.03554463, -0.05310963,  0.01546572,
        0.01381713, -0.12275856,  0.01602715, -0.07837285,  0.03044563,
        0.04087833,  0.02589903, -0.07024352, -0.061838  , -0.09467266,
        0.07097764,  0.02004194,  0.02817115,  0.02414211,  0.08

In [47]:
len(model.wv.vocab.keys())

16490

In [53]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print( "Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
        counter = counter + 1
    return reviewFeatureVecs

In [54]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000


/Users/xtreemtg888/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [59]:
trainDataVecs.shape

(25000, 300)

In [60]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

In [61]:
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [72]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors


num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  791.5672018527985 seconds.


In [68]:
word_vectors

array([[-0.11799947, -0.07569745, -0.04895984, ..., -0.00306124,
         0.15412125,  0.12473753],
       [-0.07133431, -0.03938881, -0.03463758, ...,  0.03120457,
        -0.00519378,  0.040886  ],
       [ 0.11294232,  0.00456348,  0.05030838, ...,  0.0569461 ,
        -0.02884434,  0.1159232 ],
       ...,
       [ 0.14566313, -0.04129462,  0.04122593, ..., -0.05528342,
        -0.05498267,  0.00981604],
       [ 0.22366184,  0.13692454,  0.06111889, ...,  0.03316128,
         0.05299193,  0.02905305],
       [ 0.10032341,  0.08768456,  0.0020232 , ..., -0.13776912,
        -0.04142855, -0.08648439]], dtype=float32)

In [74]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [95]:
num_clusters

3298

In [93]:
word_centroid_map

{'the': 2579,
 'and': 3173,
 'a': 2775,
 'of': 2327,
 'to': 2954,
 'is': 1556,
 'it': 933,
 'in': 762,
 'i': 1209,
 'this': 933,
 'that': 1214,
 's': 2326,
 'was': 770,
 'as': 1869,
 'with': 3276,
 'for': 2360,
 'movie': 318,
 'but': 322,
 'film': 318,
 'you': 668,
 't': 372,
 'on': 475,
 'not': 1886,
 'he': 1119,
 'are': 718,
 'his': 677,
 'have': 2282,
 'be': 3097,
 'one': 2070,
 'all': 2228,
 'at': 2034,
 'they': 668,
 'by': 2369,
 'who': 1434,
 'an': 2743,
 'from': 1472,
 'so': 1700,
 'like': 1380,
 'there': 3046,
 'her': 677,
 'or': 1551,
 'just': 699,
 'about': 1304,
 'out': 2705,
 'has': 1435,
 'if': 2405,
 'what': 1214,
 'some': 1080,
 'good': 1081,
 'can': 2524,
 'more': 1486,
 'when': 153,
 'very': 1024,
 'she': 677,
 'up': 1808,
 'no': 1692,
 'time': 757,
 'even': 1527,
 'would': 998,
 'my': 2378,
 'which': 1214,
 'their': 1834,
 'story': 1832,
 'only': 1526,
 'really': 3264,
 'see': 221,
 'had': 1435,
 'were': 718,
 'well': 387,
 'we': 668,
 'me': 1209,
 'than': 2234,
 'muc

In [87]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['veteran', 'alain', 'delon']

Cluster 1
['edges', 'spins', 'spreads', 'drowns']

Cluster 2
['distant', 'retreat', 'refugee', 'relic', 'exiled', 'province']

Cluster 3
['immature', 'uneducated']

Cluster 4
['intellectual', 'abstract', 'existential', 'academic', 'organic', 'introspective', 'ideological', 'unexplored']

Cluster 5
['shelf', 'tube', 'ebay', 'shelves', 'rack']

Cluster 6
['characterization', 'handling', 'characterizations', 'characterisation', 'characterisations']

Cluster 7
['partner', 'colleague', 'contacts', 'requests']

Cluster 8
['tight', 'stylish', 'smooth', 'slick', 'polished', 'classy', 'glossy', 'shabby', 'sleek']

Cluster 9
['pryor', 'widmark', 'gere', 'chamberlain', 'dreyfuss', 'conte', 'linklater', 'crenna']


In [88]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [89]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [91]:
train_centroids.shape

(25000, 3298)

In [100]:
train_centroids[1]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [99]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
